In [ ]:
from utils import *
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw
from IPython.display import display
from matplotlib import pyplot as plt
import os
import random

In [ ]:
# Read the XML file
with open('all/PAGE-gt-CS863/PAGE-gt/public-test/e-codices_csg-0863_017_max.xml', 'r') as file:
    xml_data = file.read()

# Parse the XML using BeautifulSoup with 'lxml-xml' parser
soup = BeautifulSoup(xml_data, 'lxml-xml')

# Find all TextLine elements with an id containing 'comment'
textlines_with_comment = soup.find_all('GraphicRegion', {'id': lambda x: x and 'region' in x})

# Extract Coords points for each matching TextLine
coordinates_by_group = {"comment":[],"body":[],"decoration":[]}
search = {"comment":["TextLine","comment"],"body":["TextLine","textline"],"decoration":["GraphicRegion","region"]}
for group in coordinates_by_group.keys():
    textlines_with_comment = soup.find_all(search[group][0], {'id': lambda x: x and search[group][1] in x})
    for textline in textlines_with_comment:
        coords_points = textline.find('Coords')['points']
        coordinate_pairs = coords_points.split()
        coordinates_list = [tuple(map(int, pair.split(','))) for pair in coordinate_pairs]
        coordinates_by_group[group].append(coordinates_list)


image_dir_path = 'all/img-CS863/img/public-test/e-codices_csg-0863_017_max.jpg'
pixel_gt_path = 'all/pixel-level-gt-CS863/pixel-level-gt/public-test/e-codices_csg-0863_017_max.png'
img = Image.open(image_dir_path)
pixel_gt = Image.open(pixel_gt_path)

colors = {"comment":( 255, 255, 0, 255),"body":(255, 0, 255, 255),"decoration":(0,255, 255, 255)}
filled_polygon_img = Image.new('RGBA', img.size, (0, 0, 0, 0))
draw_filled_polygon = ImageDraw.Draw(filled_polygon_img)

for group in coordinates_by_group.keys():
    for coords in coordinates_by_group[group]:
        polygon_coordinates = coords

        draw_filled_polygon.polygon(polygon_coordinates,outline=None,fill=colors[group])

result_img = Image.alpha_composite(img.convert('RGBA'), filled_polygon_img)
display(filled_polygon_img)


In [ ]:

arr1 = np.array(filled_polygon_img.convert('RGBA'))
arr2 = np.array(pixel_gt)
mask = arr2[:,:,0] != 0
mask_three_channels = np.repeat(mask[:, :, np.newaxis], 4, axis=2)
non_zero_intersection = (arr1!=0) & (mask_three_channels!=0)
arr1[non_zero_intersection] = 0


result_img = Image.alpha_composite(img.convert('RGBA'), filled_polygon_img)
display(Image.fromarray(arr1))
display(filled_polygon_img)

In [ ]:
def random_patch_generator(page: Page, patch_size,num_patches):
    """
    Divides an image represented by a numpy array into square patches.

    Parameters:
    - patch_size: integer, the size of each square patch in pixels.

    Returns:
    - A list of Patches, each representing a square patch of the original image.
    """
    images = []
    labels = []
    height, width, _ = page.img.shape

    for i in range(num_patches):
        start_x = random.randint(0,width-832)
        start_y = random.randint(0,height-832)
        end_x = start_x + patch_size
        end_y = start_y + patch_size

            # Slice the image array to get the current patch and add it to the list
        content = page.img[start_y:end_y,start_x:end_x, :]
        gt = page.gt[start_y:end_y,start_x:end_x]
        labels.append(gt)
        images.append(content)
    return images, labels

In [ ]:
def parse_images_from_folder(image_dir_path, xml_dir_path,pixel_dir_path):
    xml_files = os.listdir(xml_dir_path)
    image_files = os.listdir(image_dir_path)
    pixel_files = os.listdir(pixel_dir_path)
    output = []
    for i in range(len(xml_files)):
        img_file_path = image_dir_path + "/" + image_files[i]
        xml_file_path = xml_dir_path + "/" + xml_files[i]
        pixel_gt_file_path = pixel_dir_path + "/" + pixel_files[i]
        page = Page.from_file(img_file_path,xml_file_path,pixel_gt_file_path)
        output.append(page)
    return output

haha = parse_images_from_folder("all/img-CS18/img/training","all/PAGE-gt-CS18/PAGE-gt/training","all/pixel-level-gt-CS18/pixel-level-gt/training")

In [ ]:
random_patches = random_patch_generator(haha[0],832,10)

In [ ]:
display(Image.fromarray(random_patches[1].gt))

In [ ]:
def draw_grid_on_image(image, grid_x, grid_y, line_color=(0, 0, 255), line_width=20):
    """
    Draws a grid on an image and saves the result.

    Parameters:
    - image_path: Path to the original image.
    - grid_x, grid_y: The number of divisions along the x and y axes.
    - line_color: Color of the grid lines as an RGB tuple. Default is red.
    - line_width: The width of the grid lines. Default is 1.
    """
    with Image.fromarray(image) as img:
        draw = ImageDraw.Draw(img)
        width, height = img.size

        # Calculate the distance between each line
        step_x = width / grid_x
        step_y = height / grid_y

        # Draw vertical lines
        for x in range(1, grid_x):
            start = x * step_x
            draw.line((start, 0, start, height), fill=line_color, width=line_width)

        # Draw horizontal lines
        for y in range(1, grid_y):
            start = y * step_y
            draw.line((0, start, width, start), fill=line_color, width=line_width)
        #REMOVE WHEN CREATING SAMPLE PICTURES FOR REPORT
        img  = img.resize((int(img.width/4),int(img.height/4)))
        # Display the result
        display(img)

# Example usage
# draw_grid_on_image('path/to/your/image.jpg', grid_x=4, grid_y=4)

In [ ]:
draw_grid_on_image(haha[5].img,4,6)